# Assignment 5: SPARQL queries - Sergio Gonzalez Ruiz

UniProt SPARQL Endpoint: http://sparql.uniprot.org/sparql

First of all, we must select an ***endpoint*** (otherwise the query will fail) and a data ***format*** requested to the SPARQL endpoint. In order to do this, we will use **"magic instructions" ('%')**, whose documentation can be found here --> https://github.com/paulovn/sparql-kernel/blob/master/doc/magics.rst

In [2]:
%endpoint https://sparql.uniprot.org/sparql
%format JSON

Endpoint set to: https://sparql.uniprot.org/sparql
Return format: JSON

## **Q1: 1 POINT** How many protein records are in UniProt?

**Warning**: There are more than *360 millions protein records in UniProt*, so this command has quite a lot of computational complexity. Thus, it took quite a long time to process the final result.

In [2]:
PREFIX up:<http://purl.uniprot.org/core/> 

SELECT (COUNT (DISTINCT ?protein) AS ?proteincount)
WHERE 
{
    ?protein a up:Protein .
}

proteincount
360157660


Therefore, to **make the computational process more optimal**, an approximate result can be obtained, using the same script as before, but **removing the DISTINCT function**. In this way, ***we will not check duplicate protein records, but we will greatly reduce the processing time.***

In [16]:
PREFIX up:<http://purl.uniprot.org/core/> 

SELECT (COUNT (?protein) AS ?proteincount)
WHERE 
{
    ?protein a up:Protein .
}

proteincount
360157660


## **Q2: 1 POINT** How many Arabidopsis thaliana protein records are in UniProt?

In [3]:
PREFIX up:<http://purl.uniprot.org/core/> 
PREFIX taxon:<http://purl.uniprot.org/taxonomy/> 

SELECT (COUNT(DISTINCT ?protein) AS ?proteincount)
WHERE 
{
    ?protein a up:Protein .
    ?protein up:organism taxon:3702 .
}

proteincount
136782


## **Q3: 1 POINT** retrieve pictures of Arabidopsis thaliana from UniProt?

In [5]:
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?image

WHERE
{
  ?taxon a up:Taxon .
  ?taxon up:scientificName "Arabidopsis thaliana" . 
  ?taxon foaf:depiction ?image .
  ?image a foaf:Image .
}

image
https://upload.wikimedia.org/wikipedia/commons/3/39/Arabidopsis.jpg
https://upload.wikimedia.org/wikipedia/commons/thumb/6/60/Arabidopsis_thaliana_inflorescencias.jpg/800px-Arabidopsis_thaliana_inflorescencias.jpg


## **Q4: 1 POINT** What is the description of the enzyme activity of UniProt Protein Q9SZZ8

In [8]:
PREFIX up:<http://purl.uniprot.org/core/>
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos:<http://www.w3.org/2004/02/skos/core#> 
PREFIX uniprotkb:<http://purl.uniprot.org/uniprot/>

SELECT ?name ?activity
WHERE
{
    uniprotkb:Q9SZZ8 up:enzyme ?enzyme .
    ?enzyme skos:prefLabel ?name .
    ?enzyme up:activity ?act .
    ?act rdfs:label ?activity.
}


name,activity
Beta-carotene 3-hydroxylase,Beta-carotene + 4 reduced ferredoxin [iron-sulfur] cluster + 2 H(+) + 2 O(2) = zeaxanthin + 4 oxidized ferredoxin [iron-sulfur] cluster + 2 H(2)O.


## **Q5: 1 POINT** Retrieve the proteins ids, and date of submission, for proteins that have been added to UniProt this year (HINT Google for “SPARQL FILTER by date”)

Although the current date is *January 2022*, the entire year of **2021** has been considered, due to that this assignment was given to us at the end of 2021.

As I don't want it to take forever, and as the purpose of the exercise is already fulfilled, we will simply get **50 results**, because if we don't limit the search, the jupyter notebook keeps loading, until it ends up disconnecting.

In [11]:
# How to Manage Date Range SPARQL queries? --> http://docs.openlinksw.com/virtuoso/virtuosotipsandtricksmanagedaterangequery/

PREFIX up:<http://purl.uniprot.org/core/> 
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?id ?date 
WHERE
{
    ?protein a up:Protein . 
    ?protein up:created ?date .
    FILTER(?date >= xsd:date("2021-01-01") && ?date < xsd:date("2022-01-01")) . # Including 2021-01-01, but not 2022-01-01
    BIND (REPLACE(STR(?protein), "http://purl.uniprot.org/uniprot/", "") AS ?id) .
} ORDER BY ?date LIMIT 50


id,date
A0A7G2MLB1,2021-02-10
A0A7G9FM31,2021-02-10
A0A7G9FPS4,2021-02-10
A0A7G9FQN4,2021-02-10
A0A7G9FW33,2021-02-10
A0A7G9G064,2021-02-10
A0A7G9GB08,2021-02-10
A0A7G9GCB7,2021-02-10
A0A7G9GFA3,2021-02-10
A0A7G9GGH4,2021-02-10


## **Q6: 1 POINT** How many species are in the UniProt taxonomy?

In [27]:
PREFIX up:<http://purl.uniprot.org/core/> 
 
SELECT (COUNT (DISTINCT ?species) AS ?species_number)
WHERE
{
    ?species a up:Taxon .
    ?species up:rank up:Species .
}

species_number
2029846


## **Q7: 2 POINT** How many species have at least one protein record? (this might take a long time to execute, so do this one last!)

In [67]:
PREFIX up:<http://purl.uniprot.org/core/> 

SELECT (COUNT(DISTINCT ?species) AS ?species_number_with_protein_record)
WHERE 
{
    ?protein a up:Protein .
    ?protein up:organism ?species .
    ?species a up:Taxon .
    ?species up:rank up:Species .
}

species_number_with_protein_record
1057158


## **Q8: 3 points:** find the AGI codes and gene names for all Arabidopsis thaliana proteins that have a protein function annotation description that mentions “pattern formation”

In [19]:
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX taxon:<http://purl.uniprot.org/taxonomy/> 
PREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?AGI ?geneName

WHERE
{
    ?protein a up:Protein .
    ?protein up:organism taxon:3702 . # Protein variable involved with its taxon (Arabidopsis thaliana)
    ?protein up:encodedBy ?gene . # Protein variable involved with each gene
    ?protein up:annotation ?annotation. # Protein variable involved with each annotation or function
    ?gene up:locusName ?AGI . # AGI locus code of each gene
    ?gene skos:prefLabel ?geneName . # label selected (gene name) of each gene
    ?annotation a up:Function_Annotation . 
    ?annotation rdfs:comment ?text .
    FILTER CONTAINS(?text,"pattern formation") . # Applying the filter stablished above

}


AGI,geneName
At3g54220,SCR
At1g13980,GN
At4g21750,ATML1
At5g40260,SWEET8
At1g69670,CUL3B
At1g63700,YDA
At2g46710,ROPGAP3
At1g26830,CUL3A
At3g09090,DEX1
At4g37650,SHR


## From the MetaNetX metabolic networks for metagenomics database SPARQL Endpoint: https://rdf.metanetx.org/sparql (this slide deck will make it much easier for you!
https://www.metanetx.org/cgi-bin/mnxget/mnxref/MetaNetX_RDF_schema.pdf)

Therefore, we redefine the **endpoint** as follows:

In [29]:
%endpoint https://rdf.metanetx.org/sparql

Endpoint set to: https://rdf.metanetx.org/sparql

## **Q9: 4 POINTS:** what is the MetaNetX Reaction identifier (starts with “mnxr”) for the UniProt Protein uniprotkb:Q18A79

In [30]:
PREFIX mnx: <https://rdf.metanetx.org/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>

SELECT DISTINCT ?mnxr_label # With "DISTINCT" we avoid duplicate results
WHERE
{
    ?pept mnx:peptXref uniprotkb:Q18A79 .
    ?cata mnx:pept ?pept .
    ?gpr mnx:cata ?cata ;
         mnx:reac ?reaction_id .
    ?reaction_id rdfs:label ?mnxr_label .

}

mnxr_label
mnxr165934
mnxr145046c3


Thus, for the *same protein*, **2 MetaNetX Reaction identifiers** have been obtained:
* **MNXR165934**: 1 {(1->4)-alpha-D-glucosyl}(n) + 1 ADP-alpha-D-glucose <=> 1 ADP + 1 {(1->4)-alpha-D-glucosyl}(n+1) --> https://www.metanetx.org/equa_info/MNXR165934
* **MNXR145046(c3)**: 1 ADP-alpha-D-glucose <=> 1 ADP + 1 Glycogen --> https://www.metanetx.org/equa_info/MNXR145046

As it can be seen, the *first one* considers the **specific part that is involved in the reaction**: {(1->4)-alpha-D-glucosyl}(n), while the *other* considers the **Glycogen as a whole unit**.

## **Q10: 5 POINTS:** What is the official Gene ID (UniProt calls this a “mnemonic”) and the MetaNetX Reaction identifier (mnxr…..) for the protein that has “Starch synthase” catalytic activity in Clostridium difficile (taxon 272563).

In [66]:
PREFIX up: <http://purl.uniprot.org/core/>
PREFIX taxon: <http://purl.uniprot.org/taxonomy/>
PREFIX mnx: <https://rdf.metanetx.org/schema/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX uniprotkb: <http://purl.uniprot.org/uniprot/>

SELECT ?proteinRef ?Gene_ID ?MetaNetX_Reaction_ID ?activity
WHERE
{
    # First of all, we need to switch back to the Uniprot endpoint:
    service <http://sparql.uniprot.org/sparql> {
        ?protein a up:Protein .
        ?protein up:organism taxon:272563 . # Selecting a protein from Clostridium difficile (taxon 272563)
        ?protein up:mnemonic ?Gene_ID . # Official Gene ID for the protein selected
        ?protein up:classifiedWith ?goTerm . # GO term associated with the protein selected
        ?goTerm rdfs:label ?activity . # GO term of molecular activity or function
        FILTER CONTAINS(?activity, "starch synthase") # Applying the filter stablished above
        BIND (SUBSTR(STR(?protein),33) as ?proteinID)
        BIND (IRI(CONCAT(uniprotkb:,?proteinID)) as ?proteinRef)
    }
    # Switching back to the MetaNetX endpoint:  
    service <https://rdf.metanetx.org/sparql> {
        # This part is actually quite the same as in Q9:
        ?pept mnx:peptXref ?proteinRef . # We search for the protein stablished above
        ?cata mnx:pept ?pept .
        ?gpr mnx:cata ?cata ;
             mnx:reac ?reac .
        ?reac rdfs:label ?MetaNetX_Reaction_ID .
    }
} GROUP BY ?Gene_ID

proteinRef,Gene_ID,MetaNetX_Reaction_ID,activity
http://purl.uniprot.org/uniprot/Q18A79,GLGA_CLOD6,mnxr145046c3,starch synthase activity
http://purl.uniprot.org/uniprot/Q18A79,GLGA_CLOD6,mnxr165934,starch synthase activity


The ***MetaNetX Reaction identifier*** results obtained in this question coincide with those of the previous one (Q9). Therefore, the official ***Gene ID*** will be **GLGA_CLOD6** (the same for both results), as they represent **2 different reactions of the same protein**. 